In [93]:
#import package
import requests
import json
import time
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
import mysql.connector
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np

# Part 1 - a
Use previous functions to scrap each item's website in first 10 searching results pages and save them into the files.

In [338]:
# keyword: "playstation 4 slim"

def webscrap():
    
    #The url search for buy-it-now and limit the number of items to 100 per page
    url = 'https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&LH_BIN=1&_ipg=100' 
    user_agent = {'User-agent': 'Mozilla/5.0'} 
        
    # For the first 10 pages of 100 items/page
    for pgn in range(1, 11):
        time.sleep(10) 
        url_pgn = url + '&_pgn=' + str(pgn) 
        filename = 'playstation_4_slim_' + str(pgn).zfill(2) + '.htm'
        response = requests.get(url_pgn, headers = user_agent)
        with open(filename, 'w',encoding='utf-8') as file:
            file.write(response.text)          

In [339]:
webscrap() 

In [340]:
def sponsor(): 
    
    # Save all the URLs of sponsored items' pages to the file "sponsored.txt"
    for pgn in range(1,11):
        filename = 'playstation_4_slim_' + str(pgn).zfill(2) + '.htm'
        with open(filename, 'r',encoding='utf-8') as file:
            text = file.read()
            soup = BeautifulSoup(text, 'html.parser') 
            items = soup.select("li.s-item")
            for item in items:
                a = item.find("a")
                if not a:
                    continue
                
                text = item.find("span",role="text").get_text()
                # Use RegEx to define whether the item is sponsored or not
                if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',text): 
                    # Use RegEx to simplify the url
                    url_simplify = re.sub("(.*)\\?.*", r"\1", a['href'])
                    with open('sponsored.txt', 'a',encoding='utf-8') as file:
                        file.write(url_simplify + "\n")
                else :
                    url_simplify = re.sub("(.*)\\?.*", r"\1", a['href'])
                    with open('non-sponsored.txt', 'a',encoding='utf-8') as file:
                        file.write(url_simplify + "\n")

In [341]:
sponsor()

# Part 1 - b

In [342]:
# Find the directory with code script.
codepath = os.getcwd()

# Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored"
os.makedirs(codepath + "\sponsored")
os.makedirs(codepath + "\\nonsponsored")

In [343]:
# Define a function to save the html page of the url in both list for sponsored and non-sponsored
def sponsoritem(urllist,fileaddress):
    
    try:
        for item in urllist:
            time.sleep(2)            
            url = item.strip('\n')
            response = requests.get(url)

            # Name  the file as "<item-id>.htm"
            itemname = re.findall(r'.*\/(\d*)',url)
            filename = str(fileaddress) + str(itemname[0]) + ".htm"
            with open(filename,'w',encoding='utf-8') as file2:
                file2.write(response.text)
                
    except Exception as ex:
        print('error: ' + str(ex))

In [344]:
# Opens the two files in (b) and downloads each of the pages (URLs) into the folders "sponsored" 
codepath = os.getcwd()
fileaddress = codepath + "\sponsored\\"

#open sponsored.txt and stretch the list of the urls
with open('sponsored.txt', 'r',encoding='utf-8') as file1:
    urllist = file1.readlines()
    
sponsoritem(urllist,fileaddress)

In [345]:
# Opens the two files in (b) and downloads each of the pages (URLs) into the folders "non-sponsored" 
codepath = os.getcwd()
fileaddress = codepath + "\\nonsponsored\\"

#open sponsored.txt and stretch the list of the urls
with open('non-sponsored.txt', 'r',encoding='utf-8') as file1:
    urllist = file1.readlines()
    
sponsoritem(urllist,fileaddress)

# Part 1 - c

In [160]:
# Create a function to scrap all the information
def detailinfo(path,files):

    # Create empty lists
    l1=[]
    l2=[]
    l3=[]
    l4=[]
    l5=[]
    l6=[]
    l7=[]
    l8=[]
    l9=[]
    l10=[]
    l11=[]

    # Write a loop to save all the detailed information in the list
    for file in files:

        # Open the pages downloaded in (c) and parses them into a Beautifulsoup-object.
        with open(str(path) + file, 'r',encoding='utf-8') as f:
            text = f.read()
            soup = BeautifulSoup(text, 'html.parser')

            
            ####################################### Select seller name ######################################
            SellerName = soup.select("span.mbg-nw")
            # Define whether there is a null value
            if not SellerName:
                l1.append(None)
            else :
                l1.append(str(SellerName[0].string))

                
            ####################################### Select seller score ######################################
            SellerScore = soup.select("span.mbg-l a")
            # Define whether there is a null value
            if not SellerScore:
                l2.append(None)
            else :
                l2.append(str(SellerScore[0].string))

                
            ####################################### Select item price ######################################
            ItemPrice = soup.select("span.notranslate")
            # Define whether there is a null value
            if not ItemPrice:
                l3.append(None)
            else :
                l3.append(str(ItemPrice[0].string))

                
            ####################################### Select the number of items sold ######################################
            NumItemSold = soup.select("a.vi-txt-underline")
            # Define whether there is a null value
            if not NumItemSold:
                l4.append(None)
            else :
                l4.append(str(NumItemSold[0].string))

                
            ####################################### Select best offer available ######################################
            BestOfferAvail = soup.select("#boBtn_btn_parent a")
            # Define whether there is a null value
            if not BestOfferAvail:
                # Change the content to "not available"
                l5.append('Not Available')
            else :
                # Change the content to "available"
                l5.append('Available')

                
            ####################################### Select title ######################################
            Title = soup.find("h1",id = "itemTitle")
            # Define whether there is a null value
            if not Title:
                l6.append(None)
            else :
                l6.append(str(Title.get_text()))

                
            ####################################### Select returns allowed ######################################
            ReturnsAllowed = soup.find_all("span",class_="w2b-sgl")
            # Define whether there is a null value
            if not ReturnsAllowed:
                l7.append(None)
            else :
                # Loop every item in this html node
                for item in ReturnsAllowed:
                    # Check whether there is "Returns accepted", if there is, add to the list
                    match = re.search('Returns accepted',item.get_text())
                    if match:
                        l7.append("Returns accepted")
                        break
                    # Check whether there is "No returns", if there is, add to the list
                    match = re.search('No returns',item.get_text())
                    if match:
                        l7.append("No returns")
                        break
                    # If there is no "Returns accepted" or "No returns", return null value to the list
                    if (item == ReturnsAllowed[-1]):
                        l7.append(None)

                        
            ####################################### Select shipping price ######################################
            # First, check whether there is fixed shipping fee
            ShippingPrice = soup.select("span#shSummary span#fshippingCost span")
            if not ShippingPrice:
                # Second, if there isn't fixed shipping fee, check whether there is calculate shipping fee
                ShippingPrice = soup.select("span#shSummary a#e3")
                if not ShippingPrice:
                    # Third, if there isn't fixed or calculate shipping fee, check whether there is Fast N Free shipping
                    ShippingPrice = soup.select("span#shSummary span.vi-fnf-ship-txt strong")
                    if not ShippingPrice:
                        # There isn't any kind of the above three types shipping, so return null value
                        l8.append(None)
                        l9.append(None)
                    else:
                        # There is Fast N Free shipping
                        l8.append(str(ShippingPrice[0].string))
                        l9.append('No')
                else:
                    # There is calculate shipping fee
                    l8.append(None)
                    l9.append("Yes")
            else:
                # There is fixed shipping fee
                l8.append(str(ShippingPrice[0].string))
                l9.append("No")

            
            ###################### Select condition (e.g., used, new, like new, seller refurbished, ...). ##################
            Condition = soup.select("div.u-flL.condText")
            # Define whether there is a null value
            if not Condition:
                l10.append(None)
            else :
                l10.append(str(Condition[0].string))
             
            
            # Save item id
            l11.append(file[0:-4])
                
    return l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11

In [161]:
# Locate the soponsor fold and take all the files in a list
codepath = os.getcwd()
path = codepath + "\sponsored\\"
files = os.listdir(path)

# Save the scraping result into 11 sponsor lists
sl1,sl2,sl3,sl4,sl5,sl6,sl7,sl8,sl9,sl10,sl11 = detailinfo(path,files)

In [162]:
# Create a database to save all the information
Metadata_s = pd.DataFrame()
Metadata_s["SellerName"] = sl1
Metadata_s["SellerScore"] = sl2
Metadata_s["ItemPrice"] = sl3
Metadata_s["NumItemSold"] = sl4
Metadata_s["BestOfferAvail"] = sl5
Metadata_s["Title"] = sl6
Metadata_s["ReturnsAllowed"] = sl7
Metadata_s["ShippingPrice"] = sl8
Metadata_s["ShippingComputed"] = sl9
Metadata_s["Condition"] = sl10
Metadata_s["ItemID"] = sl11

In [163]:
#pd.set_option('max_row',200)
Metadata_s

,SellerName,SellerScore,ItemPrice,NumItemSold,BestOfferAvail,Title,ReturnsAllowed,ShippingPrice,ShippingComputed,Condition,ItemID
0,ezprobay99,4007,US $259.99,36 sold,Not Available,Details about Brand New Sony PlayStation 4 S...,Returns accepted,FREE,No,New,113988512101
1,monstertechbids,15302,US $239.99,21 sold,Not Available,Details about Sony Playstation 4 PS4 Slim 1T...,Returns accepted,FAST 'N FREE,No,Used,123667481471
2,donedeals30,323,US $247.00,None,Not Available,Details about Sony PlayStation 4 PS4 Slim 1T...,Returns accepted,FREE,No,Seller refurbished,124061087709
3,jalhow26,0,US $213.00,None,Available,Details about Sony PlayStation 4 Slim 1TB Co...,No returns,None,Yes,Open box,124070335389
4,captfrost,997,US $490.00,38 sold,Not Available,Details about Sony PlayStation 4 Slim or Pro...,Returns accepted,None,Yes,New,132001598506
...,...,...,...,...,...,...,...,...,...,...,...
111,alwate-48,0,US $250.00,None,Available,Details about Sony PlayStation 4 Slim 1TB- B...,No returns,None,Yes,Used,392520071460
112,itembazaar,114465,US $241.37,None,Not Available,Details about Sony PlayStation 4 Slim 1TB Co...,Returns accepted,FREE,No,Used,401786638183
113,itembazaar,114465,US $270.79,None,Not Available,Details about Sony PlayStation 4 Slim Bundle...,Returns accepted,FREE,No,Used,401871514338
114,melisswortha-0,0,US $369.00,None,Available,Details about Ps4 Slim - Days of Play Limite...,No returns,FREE,No,New,401897820918


In [244]:
# Clean the metadata

def cleandata(l3,l4,l6,l8):
    
    ################################ Clean ItemPrice column #############################

    # Make a copy
    l32 = l3.copy()

    # Create a new list to save the price unit of item price (US, C, ...)
    l3priceunit = []
    # Create a new list to save "dollar-cent" format item price
    l3priceclean = []

    for item in l32:
        if not item:
            # If there is null value, save null value to the new list too
            l3priceclean.append(None)
            l3priceunit.append(None)
        else:
            item = item.strip()
            item = item.replace(',','')
            # Use RegEx to clean the item price as "dollar-cent" format
            ItemPrice = re.sub('^[A-Z]*\s*\$(\d*)\.(\d*).*$',r'\1\2',item)
            # Save the cleaned item price in the list
            l3priceclean.append(ItemPrice)
            # Use RegEx to grasp price unit
            PriceUnit = re.sub('^([A-Z]*)\s*\$\d*\.\d*.*$',r'\1',item)
            # Save the cleaned price unit in the list
            if PriceUnit =='':
                l3priceunit.append(None)
            else:
                l3priceunit.append(PriceUnit)
            
    ################################ Clean NumItemSold column #############################

    # Make a copy
    l42 = l4.copy()

    # Create a new list to save cleaned number of item sold
    l4clean = []

    for item in l42:
        if not item:
            # If there is null value, save null value to the new list too
            l4clean.append(None)
        else:
            # Use RegEx to clean the string, only lwave the numbers and strip ','
            NumItemSold = re.sub('^(\d*),*(\d*)\s\w*$',r'\1\2',item)
            l4clean.append(NumItemSold)
            
    ################################ Clean Title column #############################

    # Make a copy
    l62 = l6.copy()

    # Create a new list to save cleaned title
    l6clean = []

    for item in l62:
        if not item:
            # If there is null value, save null value to the new list too
            l6clean.append(None)
        else:
            # Use RegEx to clean the string, strip the "Details about"
            Title = re.sub('.*?\s.*?\s{3}(.*)',r'\1',item)
            l6clean.append(Title)
            
    ################################ Clean ShippingPrice column #############################

    # Make a copy
    l82 = l8.copy()

    # Create a new list to save the price unit of shipping fees (US, C, ...)
    l8priceunit = []
    # Create a new list to save "dollar-cent" format shipping fees
    l8priceclean = []
    # Create a new list to save the free type (FREE, FAST 'N FREE, ... )
    l8freetype = []

    for item in l82:
        if not item:
            # If there is null value, save null value to the new list too
            l8priceunit.append(None)
            l8priceclean.append(None)
            l8freetype.append(None)
        else:
            if item == 'FREE':
                # If the shipping fee is free, we need to define its free type
                l8priceunit.append(None)
                l8priceclean.append('0')
                l8freetype.append(item)
            elif item == "FAST 'N FREE":
                # If the shipping fee is free, we need to define its free type
                l8priceunit.append(None)
                l8priceclean.append('0')
                l8freetype.append(item)
            elif (re.search('^[A-Z]*\s*\$\d*\.\d*.*$',item)):
                # If there is fixed shipping fees, we use RegEx to grasp the price unit (US, C, ...)
                PriceUnit = re.sub('^([A-Z]*)\s*\$\d*\.\d*.*$',r'\1',item)
                # Save the price unit to the new list
                if PriceUnit == '':
                    l8priceunit.append(None)
                else:
                    l8priceunit.append(PriceUnit)
                # If there is fixed shipping fees, we use RegEx to change its format to "dollar-cent"
                ItemPrice = re.sub('^[A-Z]*\s*\$(\d*)\.(\d*).*$',r'\1\2',item)
                # Save the new format price to the new list
                l8priceclean.append(ItemPrice)
                l8freetype.append(None)
            else:
                # Detect whether there exists the price with different format
                print("error")
    
    return l3priceunit,l3priceclean,l4clean,l6clean,l8priceunit,l8priceclean,l8freetype

In [245]:
# Clean the metadata for sponsored data
sl3priceunit,sl3priceclean,sl4clean,sl6clean,sl8priceunit,sl8priceclean,sl8freetype = cleandata(sl3,sl4,sl6,sl8)

In [246]:
# Create database with cleaned data
data_s = pd.DataFrame()
data_s["SellerName"] = sl1
data_s["SellerScore"] = sl2
data_s["ItemPriceUnit"] = sl3priceunit
data_s["ItemPrice"] = sl3priceclean
data_s["NumItemSold"] = sl4clean
data_s["BestOfferAvail"] = sl5
data_s["Title"] = sl6clean
data_s["ReturnsAllowed"] = sl7
data_s["ShippingPriceUnit"] = sl8priceunit
data_s["ShippingPrice"] = sl8priceclean
data_s["ShippingPriceFreeType"] = sl8freetype
data_s["ShippingComputed"] = sl9
data_s["Condition"] = sl10
data_s["ItemID"] = sl11

In [247]:
# Clean data frame for sponsored items
data_s["SponsorType"] = "sponsored"
data_s

,SellerName,SellerScore,ItemPriceUnit,ItemPrice,NumItemSold,BestOfferAvail,Title,ReturnsAllowed,ShippingPriceUnit,ShippingPrice,ShippingPriceFreeType,ShippingComputed,Condition,ItemID,SponsorType
0,ezprobay99,4007,US,25999,36,Not Available,Brand New Sony PlayStation 4 Slim 1TB -- Only...,Returns accepted,None,0,FREE,No,New,113988512101,sponsored
1,monstertechbids,15302,US,23999,21,Not Available,Sony Playstation 4 PS4 Slim 1TB Jet Black Cons...,Returns accepted,None,0,FAST 'N FREE,No,Used,123667481471,sponsored
2,donedeals30,323,US,24700,None,Not Available,Sony PlayStation 4 PS4 Slim 1TB ConsoleAnd Acc...,Returns accepted,None,0,FREE,No,Seller refurbished,124061087709,sponsored
3,jalhow26,0,US,21300,None,Available,Sony PlayStation 4 Slim 1TB Console - Jet Black,No returns,None,None,None,Yes,Open box,124070335389,sponsored
4,captfrost,997,US,49000,38,Not Available,Sony PlayStation 4 Slim or Pro - Customized - ...,Returns accepted,None,None,None,Yes,New,132001598506,sponsored
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,alwate-48,0,US,25000,None,Available,Sony PlayStation 4 Slim 1TB- Black w/ 2 wirele...,No returns,None,None,None,Yes,Used,392520071460,sponsored
112,itembazaar,114465,US,24137,None,Not Available,Sony PlayStation 4 Slim 1TB Console Star Wars ...,Returns accepted,None,0,FREE,No,Used,401786638183,sponsored
113,itembazaar,114465,US,27079,None,Not Available,Sony PlayStation 4 Slim Bundle PS4 Slim 500GB ...,Returns accepted,None,0,FREE,No,Used,401871514338,sponsored
114,melisswortha-0,0,US,36900,None,Available,Ps4 Slim - Days of Play Limited Edition 1TB Co...,No returns,None,0,FREE,No,New,401897820918,sponsored


In [173]:
# Locate the non-soponsored fold and take all the files in a list
codepath = os.getcwd()
path = codepath + "\\nonsponsored\\"
files = os.listdir(path)

# Save the scraping result into 11 non-sponsored lists
nl1,nl2,nl3,nl4,nl5,nl6,nl7,nl8,nl9,nl10,nl11 = detailinfo(path,files)

In [174]:
# Create a database to save all the information
Metadata_n = pd.DataFrame()
Metadata_n["SellerName"] = nl1
Metadata_n["SellerScore"] = nl2
Metadata_n["ItemPrice"] = nl3
Metadata_n["NumItemSold"] = nl4
Metadata_n["BestOfferAvail"] = nl5
Metadata_n["Title"] = nl6
Metadata_n["ReturnsAllowed"] = nl7
Metadata_n["ShippingPrice"] = nl8
Metadata_n["ShippingComputed"] = nl9
Metadata_n["Condition"] = nl10
Metadata_n["ItemID"] = nl11

In [223]:
Metadata_n

,SellerName,SellerScore,ItemPrice,NumItemSold,BestOfferAvail,Title,ReturnsAllowed,ShippingPrice,ShippingComputed,Condition,ItemID
0,vipoutlet,258288,\n\t\t\t\t\t\t\t$299.00,"1,316 sold",Not Available,Details about Sony CUH-2115B PlayStation 4 S...,Returns accepted,None,Yes,Seller refurbished,112619673851
1,e_games4u2010,15920,US $189.99,93 sold,Not Available,Details about Sony PlayStation 4 Slim 500GB ...,Returns accepted,FAST 'N FREE,No,Seller refurbished,112780300205
2,e_games4u2010,15920,US $224.99,11 sold,Not Available,Details about Sony PlayStation 4 Slim Days o...,Returns accepted,FAST 'N FREE,No,Seller refurbished,113155080430
3,vipoutlet,258288,\n\t\t\t\t\t\t\t$269.00,"1,222 sold",Not Available,Details about Sony CUH-2215A PlayStation 500...,Returns accepted,None,Yes,Seller refurbished,113326638195
4,vipoutlet,258288,\n\t\t\t\t\t\t\t$269.00,58 sold,Not Available,Details about Sony CUH-2215A PlayStation 500...,Returns accepted,None,Yes,Used,113354438187
...,...,...,...,...,...,...,...,...,...,...,...
941,br_42425,7,US $240.00,None,Available,Details about Sony PlayStation 4 Slim 1TB Bl...,No returns,None,Yes,Open box,402055198059
942,liag3448,0,US $250.00,None,Not Available,Details about Sony Playstation 4 Slim Unchar...,No returns,None,Yes,Used,402063334632
943,antpl6,4,US $125.00,None,Not Available,Details about playstation 4 slim black 1tb used,No returns,FREE,No,Used,402065097030
944,dancof-24,5,US $180.00,None,Not Available,Details about Sony PlayStation 4 Slim 1TB Bl...,No returns,None,Yes,Used,402066785154


In [248]:
# Clean the metadata for non-sponsored data
nl3priceunit,nl3priceclean,nl4clean,nl6clean,nl8priceunit,nl8priceclean,nl8freetype = cleandata(nl3,nl4,nl6,nl8)

In [249]:
# Create database with cleaned data
data_n = pd.DataFrame()
data_n["SellerName"] = nl1
data_n["SellerScore"] = nl2
data_n["ItemPriceUnit"] = nl3priceunit
data_n["ItemPrice"] = nl3priceclean
data_n["NumItemSold"] = nl4clean
data_n["BestOfferAvail"] = nl5
data_n["Title"] = nl6clean
data_n["ReturnsAllowed"] = nl7
data_n["ShippingPriceUnit"] = nl8priceunit
data_n["ShippingPrice"] = nl8priceclean
data_n["ShippingPriceFreeType"] = nl8freetype
data_n["ShippingComputed"] = nl9
data_n["Condition"] = nl10
data_n["ItemID"] = nl11

In [250]:
# Clean data frame for non-sponsored items
data_n["SponsorType"] = "non-sponsored"
data_n

,SellerName,SellerScore,ItemPriceUnit,ItemPrice,NumItemSold,BestOfferAvail,Title,ReturnsAllowed,ShippingPriceUnit,ShippingPrice,ShippingPriceFreeType,ShippingComputed,Condition,ItemID,SponsorType
0,vipoutlet,258288,None,29900,1316,Not Available,Sony CUH-2115B PlayStation 4 Slim 1TB Gaming C...,Returns accepted,None,None,None,Yes,Seller refurbished,112619673851,non-sponsored
1,e_games4u2010,15920,US,18999,93,Not Available,Sony PlayStation 4 Slim 500GB Matte Black PS4 ...,Returns accepted,None,0,FAST 'N FREE,No,Seller refurbished,112780300205,non-sponsored
2,e_games4u2010,15920,US,22499,11,Not Available,Sony PlayStation 4 Slim Days of Play Limited E...,Returns accepted,None,0,FAST 'N FREE,No,Seller refurbished,113155080430,non-sponsored
3,vipoutlet,258288,None,26900,1222,Not Available,Sony CUH-2215A PlayStation 500GB Slim System P...,Returns accepted,None,None,None,Yes,Seller refurbished,113326638195,non-sponsored
4,vipoutlet,258288,None,26900,58,Not Available,Sony CUH-2215A PlayStation 500GB Slim System P...,Returns accepted,None,None,None,Yes,Used,113354438187,non-sponsored
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,br_42425,7,US,24000,None,Available,Sony PlayStation 4 Slim 1TB Black Console,No returns,None,None,None,Yes,Open box,402055198059,non-sponsored
942,liag3448,0,US,25000,None,Not Available,Sony Playstation 4 Slim Uncharted 4 Console Bu...,No returns,None,None,None,Yes,Used,402063334632,non-sponsored
943,antpl6,4,US,12500,None,Not Available,playstation 4 slim black 1tb used,No returns,None,0,FREE,No,Used,402065097030,non-sponsored
944,dancof-24,5,US,18000,None,Not Available,Sony PlayStation 4 Slim 1TB Black Console,No returns,None,None,None,Yes,Used,402066785154,non-sponsored


In [251]:
# Concat the data for sponsored items and non-sponsored items
data = data_s.append(data_n)
data

,SellerName,SellerScore,ItemPriceUnit,ItemPrice,NumItemSold,BestOfferAvail,Title,ReturnsAllowed,ShippingPriceUnit,ShippingPrice,ShippingPriceFreeType,ShippingComputed,Condition,ItemID,SponsorType
0,ezprobay99,4007,US,25999,36,Not Available,Brand New Sony PlayStation 4 Slim 1TB -- Only...,Returns accepted,None,0,FREE,No,New,113988512101,sponsored
1,monstertechbids,15302,US,23999,21,Not Available,Sony Playstation 4 PS4 Slim 1TB Jet Black Cons...,Returns accepted,None,0,FAST 'N FREE,No,Used,123667481471,sponsored
2,donedeals30,323,US,24700,None,Not Available,Sony PlayStation 4 PS4 Slim 1TB ConsoleAnd Acc...,Returns accepted,None,0,FREE,No,Seller refurbished,124061087709,sponsored
3,jalhow26,0,US,21300,None,Available,Sony PlayStation 4 Slim 1TB Console - Jet Black,No returns,None,None,None,Yes,Open box,124070335389,sponsored
4,captfrost,997,US,49000,38,Not Available,Sony PlayStation 4 Slim or Pro - Customized - ...,Returns accepted,None,None,None,Yes,New,132001598506,sponsored
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,br_42425,7,US,24000,None,Available,Sony PlayStation 4 Slim 1TB Black Console,No returns,None,None,None,Yes,Open box,402055198059,non-sponsored
942,liag3448,0,US,25000,None,Not Available,Sony Playstation 4 Slim Uncharted 4 Console Bu...,No returns,None,None,None,Yes,Used,402063334632,non-sponsored
943,antpl6,4,US,12500,None,Not Available,playstation 4 slim black 1tb used,No returns,None,0,FREE,No,Used,402065097030,non-sponsored
944,dancof-24,5,US,18000,None,Not Available,Sony PlayStation 4 Slim 1TB Black Console,No returns,None,None,None,Yes,Used,402066785154,non-sponsored


# Part 2

In [ ]:
# Connect to the database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password=''
)

# Create a cursor
cursor = conn.cursor()

# Create 'eBay' database
cursor.execute("CREATE DATABASE eBay")

In [252]:
# Use 'eBay' database
engine = create_engine('mysql+mysqlconnector://root:@localhost:3306/eBay', echo = False)

# Create connection
con = engine.connect()

In [253]:
# Import dataframe in d into 'eBay' database
data.to_sql(name='ebay_items', con=con, if_exists='replace', index=False,
            dtype={'SellerName': sqlalchemy.types.VARCHAR(length=255), 
                   'SellerScore': sqlalchemy.types.INTEGER(),
                   'ItemPriceUnit': sqlalchemy.types.VARCHAR(length=255),
                   'ItemPrice': sqlalchemy.types.INTEGER(),
                   'NumItemSold': sqlalchemy.types.INTEGER(),
                   'BestOfferAvail': sqlalchemy.types.VARCHAR(length=255),
                   'Title': sqlalchemy.types.VARCHAR(length=255),
                   'ReturnsAllowed': sqlalchemy.types.VARCHAR(length=255),
                   'ShippingPriceUnit': sqlalchemy.types.VARCHAR(length=255),
                   'ShippingPrice': sqlalchemy.types.INTEGER(),
                   'ShippingPriceFreeType': sqlalchemy.types.VARCHAR(length=255),
                   'ShippingComputed': sqlalchemy.types.VARCHAR(length=255),
                   'Condition': sqlalchemy.types.VARCHAR(length=255),
                   'ItemID': sqlalchemy.types.VARCHAR(length=255),
                   'SponsorType': sqlalchemy.types.VARCHAR(length=255)})

In [ ]:
# If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table.

# In part d, I used None when there is missing value. When I import the dataframe, None value will be transformed as NULL automatically.


# Convert any price (item price and shipping price) into a "dollar-cent" format 
# (e.g., convert $12.34 into 1234 and $12 into 1200. Make sure the two least significant digits are cents. 
# If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

# This has been done in d part, cleandata function.

# Part 3 - a

In [256]:
# First compute mean, min, max, and sd for numerical column, grouped by "sponsor/non-sponsor" and "condition".

sql = '''SELECT SponsorType, `Condition`, avg(SellerScore), min(SellerScore), max(SellerScore), stddev(SellerScore),
avg(ItemPrice), min(ItemPrice), max(ItemPrice), stddev(ItemPrice), avg(NumItemSold), min(NumItemSold), max(NumItemSold),
stddev(NumItemSold), avg(ShippingPrice), min(ShippingPrice), max(ShippingPrice), stddev(ShippingPrice)
FROM ebay.ebay_items 
GROUP BY SponsorType, `Condition` 
ORDER BY SponsorType,`Condition`;'''

myresult = pd.read_sql(sql, con=con)
myresult

,SponsorType,Condition,avg(SellerScore),min(SellerScore),max(SellerScore),stddev(SellerScore),avg(ItemPrice),min(ItemPrice),max(ItemPrice),stddev(ItemPrice),avg(NumItemSold),min(NumItemSold),max(NumItemSold),stddev(NumItemSold),avg(ShippingPrice),min(ShippingPrice),max(ShippingPrice),stddev(ShippingPrice)
0,non-sponsored,Manufacturer refurbished,452612.0000,0,2263005,905196.500122,21359.8000,19000,25000,2148.940241,5.0000,5.0,5.0,0.000000,0.0000,0,0,0.000000
1,non-sponsored,New,9516.3481,-1,255864,30588.922277,32015.3717,16100,126999,10908.586923,53.0625,1.0,1565.0,206.876917,317.4627,0,4854,878.825387
2,non-sponsored,Open box,24731.5385,0,638221,106169.103119,26844.4154,3000,75000,9702.534173,6.5000,1.0,15.0,5.172040,572.0000,0,3000,985.169379
3,non-sponsored,Seller refurbished,221190.4444,0,2263005,621584.757274,22376.8889,6495,36995,5936.686491,232.2500,1.0,1316.0,464.837270,278.2083,0,2999,762.454588
4,non-sponsored,Used,48848.4212,0,258288,74222.369347,22482.4810,9300,48999,4798.463291,9.2308,1.0,58.0,15.028966,395.1790,0,4660,812.103335
5,sponsored,Manufacturer refurbished,196.6667,3,531,237.397463,26133.0000,19999,33900,5791.342216,3.0000,3.0,3.0,0.000000,0.0000,0,0,0.000000
6,sponsored,New,41745.8913,0,916562,157109.570765,33476.6739,22100,59999,10229.275284,145.7222,1.0,1896.0,430.551559,344.0588,0,4999,1123.987150
7,sponsored,Open box,2922.6000,0,14563,5820.212457,25260.0000,19500,33500,5017.011062,NaN,NaN,NaN,NaN,1500.0000,1500,1500,0.000000
8,sponsored,Seller refurbished,15084.7500,0,41442,16973.055402,22999.5000,19999,25300,2132.369633,851.0000,4.0,1698.0,847.000000,0.0000,0,0,0.000000
9,sponsored,Used,16826.3621,0,114465,37008.700499,28448.7069,16000,55351,7526.793639,16.0000,11.0,21.0,5.000000,1297.5000,0,7000,2360.261959


In [257]:
# Second compute mean, min, max, and sd for binary column, grouped by "sponsor/non-sponsor" and "condition".

sql = '''SELECT SponsorType, `Condition`, avg(CASE WHEN BestOfferAvail = "Not Available" THEN 0 WHEN BestOfferAvail = "Available" THEN 1 END) as avgBestOfferAvail,
min(CASE WHEN BestOfferAvail = "Not Available" THEN 0 WHEN BestOfferAvail = "Available" THEN 1 END) as minBestOfferAvail,
max(CASE WHEN BestOfferAvail = "Not Available" THEN 0 WHEN BestOfferAvail = "Available" THEN 1 END) as maxBestOfferAvail,
stddev(CASE WHEN BestOfferAvail = "Not Available" THEN 0 WHEN BestOfferAvail = "Available" THEN 1 END) as sdBestOfferAvail,
avg(CASE WHEN ReturnsAllowed = "No returns" THEN 0 WHEN ReturnsAllowed = "Returns accepted" THEN 1 END) as avgReturnsAllowed,
min(CASE WHEN ReturnsAllowed = "No returns" THEN 0 WHEN ReturnsAllowed = "Returns accepted" THEN 1 END) as minReturnsAllowed,
max(CASE WHEN ReturnsAllowed = "No returns" THEN 0 WHEN ReturnsAllowed = "Returns accepted" THEN 1 END) as maxReturnsAllowed,
stddev(CASE WHEN ReturnsAllowed = "No returns" THEN 0 WHEN ReturnsAllowed = "Returns accepted" THEN 1 END) as sdReturnsAllowed,
avg(CASE WHEN ShippingComputed = "No" THEN 0 WHEN ShippingComputed = "Yes" THEN 1 END) as avgShippingComputed,
min(CASE WHEN ShippingComputed = "No" THEN 0 WHEN ShippingComputed = "Yes" THEN 1 END) as minShippingComputed,
max(CASE WHEN ShippingComputed = "No" THEN 0 WHEN ShippingComputed = "Yes" THEN 1 END) as maxShippingComputed,
stddev(CASE WHEN ShippingComputed = "No" THEN 0 WHEN ShippingComputed = "Yes" THEN 1 END) as sdShippingComputed
FROM ebay.ebay_items
GROUP BY SponsorType, `Condition`
ORDER BY SponsorType,`Condition`;'''

myresult = pd.read_sql(sql, con=con)
myresult

,SponsorType,Condition,avgBestOfferAvail,minBestOfferAvail,maxBestOfferAvail,sdBestOfferAvail,avgReturnsAllowed,minReturnsAllowed,maxReturnsAllowed,sdReturnsAllowed,avgShippingComputed,minShippingComputed,maxShippingComputed,sdShippingComputed
0,non-sponsored,Manufacturer refurbished,0.8000,0,1,0.400000,0.4000,0,1,0.489898,0.8000,0,1,0.400000
1,non-sponsored,New,0.3097,0,1,0.462384,0.5168,0,1,0.499717,0.2296,0,1,0.420580
2,non-sponsored,Open box,0.5692,0,1,0.495184,0.2188,0,1,0.413399,0.7167,0,1,0.450617
3,non-sponsored,Seller refurbished,0.2500,0,1,0.433013,0.5714,0,1,0.494872,0.3143,0,1,0.464231
4,non-sponsored,Used,0.4491,0,1,0.497403,0.4819,0,1,0.499671,0.3179,0,1,0.465658
5,sponsored,Manufacturer refurbished,1.0000,1,1,0.000000,0.3333,0,1,0.471405,0.6667,0,1,0.471405
6,sponsored,New,0.2609,0,1,0.439109,0.6739,0,1,0.468780,0.2444,0,1,0.429757
7,sponsored,Open box,0.8000,0,1,0.400000,0.2000,0,1,0.400000,0.7500,0,1,0.433013
8,sponsored,Seller refurbished,0.2500,0,1,0.433013,0.7500,0,1,0.433013,0.2500,0,1,0.433013
9,sponsored,Used,0.5345,0,1,0.498810,0.5000,0,1,0.500000,0.3214,0,1,0.467025


In [258]:
# Third, compute the count of each category level for the other column. I skip the title column as 
# it is meaningless to see the frequency of title.

# See the frequency of SellerName
sql = '''SELECT SponsorType, `Condition`, SellerName, count(SellerName)
FROM ebay.ebay_items 
GROUP BY SponsorType, `Condition`, SellerName 
ORDER BY SponsorType,`Condition`, count(SellerName) desc;'''

myresult = pd.read_sql(sql, con=con)
myresult

,SponsorType,Condition,SellerName,count(SellerName)
0,non-sponsored,Manufacturer refurbished,ahlehhx,1
1,non-sponsored,Manufacturer refurbished,146110-59,1
2,non-sponsored,Manufacturer refurbished,only3551,1
3,non-sponsored,Manufacturer refurbished,martiramire9,1
4,non-sponsored,Manufacturer refurbished,decluttr_store,1
...,...,...,...,...
768,sponsored,Used,anhi-4493,1
769,sponsored,Used,kids_16,1
770,sponsored,Used,monstertechbids,1
771,sponsored,Used,princes-9700,1


In [259]:
# See the frequency of ItemPriceUnit
sql = '''SELECT SponsorType, `Condition`, ItemPriceUnit, count(ItemPriceUnit)
FROM ebay.ebay_items 
GROUP BY SponsorType, `Condition`, ItemPriceUnit
ORDER BY SponsorType,`Condition`, count(ItemPriceUnit) desc;'''

myresult = pd.read_sql(sql, con=con)
myresult

,SponsorType,Condition,ItemPriceUnit,count(ItemPriceUnit)
0,non-sponsored,Manufacturer refurbished,US,5
1,non-sponsored,New,US,334
2,non-sponsored,New,None,0
3,non-sponsored,Open box,US,64
4,non-sponsored,Open box,None,0
5,non-sponsored,Seller refurbished,US,34
6,non-sponsored,Seller refurbished,None,0
7,non-sponsored,Used,US,500
8,non-sponsored,Used,None,0
9,sponsored,Manufacturer refurbished,US,3


In [260]:
# See the frequency of ShippingPriceUnit
sql = '''SELECT SponsorType, `Condition`, ShippingPriceUnit, count(ShippingPriceUnit)
FROM ebay.ebay_items 
GROUP BY SponsorType, `Condition`, ShippingPriceUnit
ORDER BY SponsorType,`Condition`, count(ShippingPriceUnit) desc;'''

myresult = pd.read_sql(sql, con=con)
myresult

,SponsorType,Condition,ShippingPriceUnit,count(ShippingPriceUnit)
0,non-sponsored,Manufacturer refurbished,None,0
1,non-sponsored,New,None,0
2,non-sponsored,Open box,None,0
3,non-sponsored,Seller refurbished,None,0
4,non-sponsored,Used,None,0
5,sponsored,Manufacturer refurbished,None,0
6,sponsored,New,None,0
7,sponsored,Open box,None,0
8,sponsored,Seller refurbished,None,0
9,sponsored,Used,C,7


In [261]:
# See the frequency of ShippingPriceFreeType
sql = '''SELECT SponsorType, `Condition`, ShippingPriceFreeType, count(ShippingPriceFreeType)
FROM ebay.ebay_items 
GROUP BY SponsorType, `Condition`, ShippingPriceFreeType
ORDER BY SponsorType,`Condition`, count(ShippingPriceFreeType) desc;'''

myresult = pd.read_sql(sql, con=con)
myresult

,SponsorType,Condition,ShippingPriceFreeType,count(ShippingPriceFreeType)
0,non-sponsored,Manufacturer refurbished,FREE,1
1,non-sponsored,Manufacturer refurbished,None,0
2,non-sponsored,New,FREE,156
3,non-sponsored,New,FAST 'N FREE,55
4,non-sponsored,New,None,0
5,non-sponsored,Open box,FREE,10
6,non-sponsored,Open box,FAST 'N FREE,2
7,non-sponsored,Open box,None,0
8,non-sponsored,Seller refurbished,FREE,16
9,non-sponsored,Seller refurbished,FAST 'N FREE,5


In [262]:
# See the frequency of Condition
sql = '''SELECT SponsorType, `Condition`, count(`Condition`)
FROM ebay.ebay_items 
GROUP BY SponsorType, `Condition`
ORDER BY SponsorType,`Condition`, count(`Condition`) desc;'''

myresult = pd.read_sql(sql, con=con)
myresult

,SponsorType,Condition,count(`Condition`)
0,non-sponsored,Manufacturer refurbished,5
1,non-sponsored,New,339
2,non-sponsored,Open box,65
3,non-sponsored,Seller refurbished,36
4,non-sponsored,Used,501
5,sponsored,Manufacturer refurbished,3
6,sponsored,New,46
7,sponsored,Open box,5
8,sponsored,Seller refurbished,4
9,sponsored,Used,58


# Part 3 - b

In [ ]:
# As the frequency of condition shows, "New" and "Used" conditions are two main category, 
# so I mainly focus on New and Used types.

# For seller score, Sponsored items in new condition have higher average seller score, 41745.8913, than the that of non-sponsored 
# items in new condition, 9516.3481. However, large standard deviation in sponsored new items should be considerated.
# Non-sponsored used items have almost twice the seller score as that of sponsored items.

# For item price, non-sponsored items have lower average item price than the correponding category pf soponsored items 
# except for open box category which can be neglected.

# For the number of item sold, non-sponsored items have lower average numbers of item sold than the corrsponding category 
# of sponsored items.

# For shipping price, there isn't much diffrence between non-sponsored new items and sponsored new items.
# However, non-sponsored used items have much lower average shipping price than sponsored used items.

# For best offer, non-sponsored new items have rough 0.05 higher average proportion than that of sponsored items.
# However, sponsored used items have higher average proportion than that of non-sponsored items, about 0.085.

# For returns, non-sponsored items have lower average returns allowed proportion than that of sponsored items.

# For calculated shipping fees, there isn't much difference between the proportions of sponsored items and non-sponsored items.


In [265]:
# As the analysis above shows, there are a lot of difference pattern between sponsor and non-sponsor items,
# which means a lot of variables could be used as predictors to capture the difference patterns.

# I would choose seller score, item price, numbers of item sold, shipping price, best offer available, returns allowed
# and condition as the predictors.